This code is a template for usage of machine learning model in PM2.5 stimation with Aerosol Optical Depth and meteorological data. We used it for different datasets in different cities.

NEW CODE

In [ ]:
"""
PART 1: DATA PREPARATION & EXPLORATORY ANALYSIS
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib as mpl
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

# Set publication-quality plot settings
plt.style.use('seaborn-whitegrid')
mpl.rcParams['figure.figsize'] = (12, 8)
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['legend.title_fontsize'] = 14
mpl.rcParams['figure.dpi'] = 150
colors = plt.cm.tab10(np.linspace(0, 1, 10))

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 1. Load and inspect data
print("Loading and inspecting data...")
df = pd.read_csv("../data/input.csv", parse_dates=['date'], index_col="date")
print(f"Dataset shape: {df.shape}")
print(f"Available features: {df.columns.tolist()}")

# Remove Season, Month, PM25lag and PM25lag2 columns if they exist
columns_to_remove = ['S', 'M', 'PM25lag', 'PM25lag2']
for col in columns_to_remove:
    if col in df.columns:
        df = df.drop(col, axis=1)
        print(f"Removed {col} from dataset")

# 2. Check for missing values
missing_data = df.isnull().sum()
print("\nMissing values per feature:")
print(missing_data[missing_data > 0])
print(f"Total missing values: {df.isnull().sum().sum()}")

# Create a heatmap of missing values
plt.figure(figsize=(14, 8))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values in Dataset')
plt.tight_layout()
plt.savefig('../figures/missing_values.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. Descriptive statistics
print("\nData summary statistics:")
desc_stats = df.describe().transpose()
desc_stats['missing'] = df.isnull().sum()
desc_stats['missing_percent'] = (df.isnull().sum() / len(df)) * 100
print(desc_stats)

# Save descriptive statistics to CSV for paper
desc_stats.to_csv('../results/descriptive_statistics.csv')

# 4. Outlier detection and removal
def detect_and_remove_outliers(df, columns, method='zscore', threshold=3):
    """
    Detect and remove outliers from specified columns using different methods
    
    Parameters:
    -----------
    df : pandas DataFrame
        The input dataframe
    columns : list
        List of column names to check for outliers
    method : str
        Method to use for outlier detection ('zscore', 'iqr', or 'percentile')
    threshold : float
        Threshold for z-score method (default=3)
        
    Returns:
    --------
    df_clean : pandas DataFrame
        DataFrame with outliers removed
    outlier_info : dict
        Information about removed outliers
    """
    df_clean = df.copy()
    outlier_info = {}
    
    for col in columns:
        if col not in df.columns:
            continue
            
        # Skip columns with all NaN values
        if df[col].isna().all():
            continue
            
        outlier_info[col] = {'count_before': df[col].count()}
        
        if method == 'zscore':
            # Z-score method
            z_scores = np.abs(stats.zscore(df[col].dropna()))
            outliers = df[col].dropna()[z_scores > threshold].index
            df_clean.loc[outliers, col] = np.nan
            
        elif method == 'iqr':
            # IQR method
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            df_clean.loc[outliers, col] = np.nan
            
        elif method == 'percentile':
            # Percentile method (remove top and bottom 1%)
            lower_bound = df[col].quantile(0.01)
            upper_bound = df[col].quantile(0.99)
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            df_clean.loc[outliers, col] = np.nan
            
        outlier_info[col]['count_after'] = df_clean[col].count()
        outlier_info[col]['outliers_removed'] = outlier_info[col]['count_before'] - outlier_info[col]['count_after']
        outlier_info[col]['percent_removed'] = (outlier_info[col]['outliers_removed'] / outlier_info[col]['count_before']) * 100
        
    return df_clean, outlier_info

# Detect and remove outliers
print("\nDetecting and removing outliers...")
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
df_clean, outlier_info = detect_and_remove_outliers(df, numerical_columns, method='iqr')

# Print outlier removal summary
print("\nOutlier removal summary:")
for col, info in outlier_info.items():
    if 'outliers_removed' in info:
        print(f"{col}: Removed {info['outliers_removed']} outliers ({info['percent_removed']:.2f}%)")

# 5. Data distribution visualization
def plot_distributions(df, features):
    """Create distribution plots for specified features"""
    n_features = len(features)
    n_cols = 2
    n_rows = (n_features + 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14, 4*n_rows))
    axes = axes.flatten()
    
    for i, feature in enumerate(features):
        if i < len(axes):
            # Histogram with KDE
            sns.histplot(df[feature].dropna(), kde=True, ax=axes[i], color=colors[i % 10])
            axes[i].set_title(f'Distribution of {feature}')
            axes[i].set_xlabel(feature)
            axes[i].set_ylabel('Frequency')
            
            # Add mean and median lines
            mean_val = df[feature].mean()
            median_val = df[feature].median()
            axes[i].axvline(mean_val, color='red', linestyle='--', 
                           label=f'Mean: {mean_val:.2f}')
            axes[i].axvline(median_val, color='green', linestyle='-.',
                           label=f'Median: {median_val:.2f}')
            axes[i].legend()
    
    # Hide any unused subplots
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    plt.savefig(f'../figures/feature_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()

# Select key features for distribution analysis
key_features = ['PM25', 'AOD', 'T2M', 'BL', 'U10', 'V10']
plot_distributions(df_clean, key_features)

# 6. Focus on rows with both PM2.5 and AOD data
print("\nAnalyzing PM2.5 and AOD relationship...")
df_valid = df_clean.dropna(subset=['PM25', 'AOD']).copy()
print(f"Valid data (no NaN in PM2.5 or AOD): {df_valid.shape}")

# Calculate correlation coefficient
corr_coef = df_valid['PM25'].corr(df_valid['AOD'])

# Visualize PM2.5 vs AOD scatter plot with correlation
plt.figure(figsize=(10, 8))
plt.scatter(df_valid['AOD'], df_valid['PM25'], alpha=0.5, color=colors[0])
plt.xlabel('AOD')
plt.ylabel('PM2.5 (μg/m³)')
plt.title(f'PM2.5 vs AOD (r = {corr_coef:.3f})')
plt.grid(True)
plt.savefig('../figures/pm25_aod_scatter.png', dpi=300, bbox_inches='tight')
plt.show()

# 7. Correlation analysis
# Calculate correlation matrix
corr_matrix = df_valid.corr()

# Create heatmap
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr_matrix, mask=mask, cmap=cmap, annot=True, fmt='.2f', 
           center=0, square=True, linewidths=.5)
plt.title('Correlation Matrix of Features')
plt.tight_layout()
plt.savefig('../figures/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Focus on PM2.5 correlations
pm25_corr = corr_matrix['PM25'].sort_values(ascending=False)
print("\nFeatures correlation with PM2.5:")
print(pm25_corr)

# Create horizontal bar plot of correlations with PM2.5
plt.figure(figsize=(10, 8))
pm25_corr.drop('PM25').plot(kind='barh', color=[colors[i % 10] for i in range(len(pm25_corr)-1)])
plt.title('Feature Correlation with PM2.5')
plt.xlabel('Pearson Correlation Coefficient')
plt.grid(True)
plt.tight_layout()
plt.savefig('../figures/pm25_correlations.png', dpi=300, bbox_inches='tight')
plt.show()

# 8. Time series analysis (if DATE is available)
if isinstance(df_valid.index, pd.DatetimeIndex):
    # Resample to monthly averages
    monthly_data = df_valid.resample('M').mean()
    
    # Calculate monthly correlations
    monthly_corr = monthly_data['PM25'].corr(monthly_data['AOD'])
    
    # Plot PM2.5 and AOD time series
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12), sharex=True)
    
    # PM2.5 time series
    ax1.plot(monthly_data.index, monthly_data['PM25'], 'o-', color=colors[0], linewidth=2)
    ax1.set_ylabel('PM2.5 (μg/m³)')
    ax1.set_title(f'Monthly Average PM2.5 Concentration')
    ax1.grid(True)
    
    # AOD time series
    ax2.plot(monthly_data.index, monthly_data['AOD'], 'o-', color=colors[1], linewidth=2)
    ax2.set_ylabel('AOD')
    ax2.set_title(f'Monthly Average AOD (r = {monthly_corr:.3f})')
    ax2.set_xlabel('Date')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('../figures/time_series_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Extract seasonal patterns
    monthly_data['Month'] = monthly_data.index.month
    seasonal_data = monthly_data.groupby('Month').mean()
    
    # Calculate seasonal correlations
    seasonal_corr = seasonal_data['PM25'].corr(seasonal_data['AOD'])
    
    # Plot seasonal patterns
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    ax1.plot(seasonal_data.index, seasonal_data['PM25'], 'o-', color=colors[0], linewidth=2)
    ax1.set_xticks(range(1, 13))
    ax1.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax1.set_ylabel('PM2.5 (μg/m³)')
    ax1.set_title('Seasonal Pattern of PM2.5')
    ax1.grid(True)
    
    ax2.plot(seasonal_data.index, seasonal_data['AOD'], 'o-', color=colors[1], linewidth=2)
    ax2.set_xticks(range(1, 13))
    ax2.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax2.set_ylabel('AOD')
    ax2.set_title(f'Seasonal Pattern of AOD (r = {seasonal_corr:.3f})')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('../figures/seasonal_patterns.png', dpi=300, bbox_inches='tight')
    plt.show()

# 9. Save data for further analysis
df_valid.to_csv('../data/cleaned_data.csv')
print("\nPart 1 completed. Data saved for further analysis.")

In [ ]:
"""
PART 2: FEATURE ENGINEERING & PREPROCESSING
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from datetime import datetime
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 1. Load the cleaned data from Part 1
print("Loading cleaned data...")
df_cleaned = pd.read_csv('../data/cleaned_data.csv', parse_dates=['date'], index_col='date')
print(f"Cleaned data shape: {df_cleaned.shape}")

# 2. PM2.5/AOD Ratio Analysis (important for research papers)
df_cleaned['PM25_AOD_Ratio'] = df_cleaned['PM25'] / df_cleaned['AOD']

# Remove infinite values from division by zero
df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)

# Calculate ratio statistics before outlier removal
ratio_stats_before = df_cleaned['PM25_AOD_Ratio'].describe()
print("\nPM2.5/AOD Ratio Statistics (before outlier removal):")
print(ratio_stats_before)

# Outlier removal using Z-score method
print("\nRemoving outliers...")
# Function to remove outliers using z-score
def remove_outliers(df, columns, threshold=3):
    df_clean = df.copy()
    for col in columns:
        if col in df.columns:
            z_scores = stats.zscore(df_clean[col].dropna())
            abs_z_scores = np.abs(z_scores)
            filtered_entries = abs_z_scores < threshold
            df_clean.loc[df_clean[col].dropna().index[~filtered_entries], col] = np.nan
            print(f"Removed {(~filtered_entries).sum()} outliers from {col}")
    return df_clean

# Apply outlier removal to key columns
outlier_columns = ['PM25', 'AOD', 'PM25_AOD_Ratio', 'T2M', 'BL', 'SP', 'D2M']
outlier_columns = [col for col in outlier_columns if col in df_cleaned.columns]
df_cleaned = remove_outliers(df_cleaned, outlier_columns)

# Calculate ratio statistics after outlier removal
ratio_stats_after = df_cleaned['PM25_AOD_Ratio'].describe()
print("\nPM2.5/AOD Ratio Statistics (after outlier removal):")
print(ratio_stats_after)

# Visualize the ratio distribution after outlier removal
plt.figure(figsize=(12, 8))
sns.histplot(df_cleaned['PM25_AOD_Ratio'].dropna(), kde=True, bins=30)
plt.axvline(df_cleaned['PM25_AOD_Ratio'].mean(), color='red', linestyle='--', 
           label=f'Mean: {df_cleaned["PM25_AOD_Ratio"].mean():.2f}')
plt.axvline(df_cleaned['PM25_AOD_Ratio'].median(), color='green', linestyle='-.',
           label=f'Median: {df_cleaned["PM25_AOD_Ratio"].median():.2f}')
plt.xlabel('PM2.5/AOD Ratio')
plt.ylabel('Frequency')
plt.title('Distribution of PM2.5/AOD Ratio (After Outlier Removal)')
plt.grid(True)
plt.legend()
plt.savefig('../figures/pm25_aod_ratio_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. Feature Engineering
# Add temporal features if DATE is the index
if isinstance(df_cleaned.index, pd.DatetimeIndex):
    print("\nAdding temporal features...")
    # Extract temporal features
    df_cleaned['DayOfYear'] = df_cleaned.index.dayofyear

# 4. Analyze PM2.5-AOD relationship
# Calculate correlation for the scatter plot
overall_corr = df_cleaned['PM25'].corr(df_cleaned['AOD'])

# Plot PM2.5 vs AOD
plt.figure(figsize=(14, 10))
plt.scatter(df_cleaned['AOD'], df_cleaned['PM25'], alpha=0.6)
plt.xlabel('AOD')
plt.ylabel('PM2.5 (μg/m³)')
plt.title(f'PM2.5 vs AOD Relationship (Overall r = {overall_corr:.3f})')
plt.grid(True)
plt.savefig('../figures/pm25_aod_relationship.png', dpi=300, bbox_inches='tight')
plt.show()

# 5. Interaction features
print("\nCreating interaction features...")
# AOD × meteorological interactions
df_cleaned['AOD_T2M'] = df_cleaned['AOD'] * df_cleaned['T2M']
df_cleaned['AOD_BL'] = df_cleaned['AOD'] * df_cleaned['BL']
df_cleaned['AOD_RH'] = df_cleaned['AOD'] * df_cleaned['D2M'] if 'D2M' in df_cleaned.columns else 0

# Wind components interaction (for wind speed)
if 'U10' in df_cleaned.columns and 'V10' in df_cleaned.columns:
    df_cleaned['WindSpeed'] = np.sqrt(df_cleaned['U10']**2 + df_cleaned['V10']**2)
    df_cleaned['AOD_WindSpeed'] = df_cleaned['AOD'] * df_cleaned['WindSpeed']

# 6. Feature selection
# Select features for modeling (adjust based on your dataset)
potential_features = ['AOD', 'T2M', 'BL', 'SP', 'D2M', 'U10', 'V10', 
                     'WindSpeed', 'AOD_T2M', 'AOD_BL']

# Filter to only include features that exist in the dataframe
features = [f for f in potential_features if f in df_cleaned.columns]
target = 'PM25'

print(f"\nSelected features for modeling: {features}")

# 7. Data split
print("\nSplitting data into training and testing sets...")
X = df_cleaned[features].copy()
y = df_cleaned[target].copy()

# Handle any remaining missing values
X.fillna(X.mean(), inplace=True)
print(f"Features shape after handling missing values: {X.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

# 8. Feature scaling
print("\nScaling features...")
# RobustScaler is good for data with outliers
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames for better interpretability
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

# 9. Save processed data for modeling
# Create a dictionary with all necessary data for modeling
processed_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'X_train_scaled': X_train_scaled,
    'X_test_scaled': X_test_scaled,
    'features': features,
    'scaler': scaler
}

# Save using numpy (more reliable for objects like the scaler)
np.save('../data/processed_data.npy', processed_data, allow_pickle=True)
print("\nPart 2 completed. Processed data saved for modeling.")

In [ ]:
"""
PART 3: MODEL DEVELOPMENT & EVALUATION
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, RandomizedSearchCV, learning_curve
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('seaborn-whitegrid')
colors = plt.cm.tab10(np.linspace(0, 1, 10))

# 1. Load processed data
print("Loading processed data...")
processed_data = np.load('../data/processed_data.npy', allow_pickle=True).item()

X_train = processed_data['X_train']
X_test = processed_data['X_test']
y_train = processed_data['y_train']
y_test = processed_data['y_test']
X_train_scaled = processed_data['X_train_scaled']
X_test_scaled = processed_data['X_test_scaled']
features = processed_data['features']
scaler = processed_data['scaler']

# Check for NaN values in target variables and handle them
print("Checking for NaN values in the data...")
if y_train.isna().any():
    print(f"Found {y_train.isna().sum()} NaN values in y_train. Removing rows with NaN values...")
    # Create a mask for non-NaN values in y_train
    train_mask = ~y_train.isna()
    X_train = X_train[train_mask]
    X_train_scaled = X_train_scaled[train_mask]
    y_train = y_train[train_mask]
    print(f"After removing NaN values: {len(y_train)} training samples remaining")

if y_test.isna().any():
    print(f"Found {y_test.isna().sum()} NaN values in y_test. Removing rows with NaN values...")
    # Create a mask for non-NaN values in y_test
    test_mask = ~y_test.isna()
    X_test = X_test[test_mask]
    X_test_scaled = X_test_scaled[test_mask]
    y_test = y_test[test_mask]
    print(f"After removing NaN values: {len(y_test)} test samples remaining")

print(f"Training data shape: {X_train.shape}")
print(f"Features: {features}")

# 2. Define models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(kernel='rbf', C=1.0, epsilon=0.1),
    'KNN': KNeighborsRegressor(n_neighbors=15)  # Increased n_neighbors to reduce overfitting
}

# 3. Create AOD-only model for baseline comparison
print("\nCreating AOD-only baseline model...")
X_aod_train = X_train[['AOD']].values.reshape(-1, 1)
X_aod_test = X_test[['AOD']].values.reshape(-1, 1)

aod_model = LinearRegression()
aod_model.fit(X_aod_train, y_train)
aod_pred = aod_model.predict(X_aod_test)

aod_mae = mean_absolute_error(y_test, aod_pred)
aod_rmse = np.sqrt(mean_squared_error(y_test, aod_pred))
aod_r2 = r2_score(y_test, aod_pred)

print(f"AOD-only model performance:")
print(f"  MAE: {aod_mae:.2f}")
print(f"  RMSE: {aod_rmse:.2f}")
print(f"  R²: {aod_r2:.4f}")
print(f"  AOD coefficient: {aod_model.coef_[0]:.4f}")
print(f"  Intercept: {aod_model.intercept_:.4f}")

# 4. Model evaluation function
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    """Train and evaluate a model, returning performance metrics"""
    # Train model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Return metrics and predictions
    return {
        'model': model,
        'model_name': model_name,
        'train_mae': train_mae,
        'train_rmse': train_rmse,
        'train_r2': train_r2,
        'test_mae': test_mae,
        'test_rmse': test_rmse,
        'test_r2': test_r2,
        'y_pred': y_test_pred
    }

# 5. Perform 10-fold cross-validation for all models
print("\nPerforming 10-fold cross-validation for all models...")
cv = KFold(n_splits=10, shuffle=True, random_state=42)
cv_results = []

for name, model in models.items():
    print(f"Cross-validating {name}...")
    
    # R² scores
    r2_scores = cross_val_score(model, X_train_scaled, y_train, 
                              cv=cv, scoring='r2', n_jobs=-1)
    
    # RMSE scores (negative converted to positive)
    rmse_scores = -cross_val_score(model, X_train_scaled, y_train, 
                                 cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
    
    # MAE scores (negative converted to positive)
    mae_scores = -cross_val_score(model, X_train_scaled, y_train, 
                                cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
    
    cv_results.append({
        'Model': name,
        'R² (mean)': r2_scores.mean(),
        'R² (std)': r2_scores.std(),
        'RMSE (mean)': rmse_scores.mean(),
        'RMSE (std)': rmse_scores.std(),
        'MAE (mean)': mae_scores.mean(),
        'MAE (std)': mae_scores.std()
    })
    
    print(f"  Mean R²: {r2_scores.mean():.4f} ± {r2_scores.std():.4f}")
    print(f"  Mean RMSE: {rmse_scores.mean():.2f} ± {rmse_scores.std():.2f}")

# Create DataFrame for CV results
cv_results_df = pd.DataFrame(cv_results)
cv_results_df = cv_results_df.sort_values('R² (mean)', ascending=False).reset_index(drop=True)
cv_results_df.to_csv('../results/cross_validation_results.csv', index=False)

print("\nCross-validation results:")
print(cv_results_df)

# 6. Hyperparameter tuning for all models
print("\nPerforming hyperparameter optimization for all models...")

# Define parameter grids for each model
param_grids = {
    'Linear Regression': None,  # No hyperparameters to tune
    
    'Random Forest': {
        'n_estimators': [100, 200, 300, 500],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2']
    },
    
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300, 500],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7, 9],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'subsample': [0.8, 0.9, 1.0]
    },
    
    'XGBoost': {
        'n_estimators': [100, 200, 300, 500],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7, 9],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'gamma': [0, 0.1, 0.2],
        'reg_alpha': [0, 0.1, 1.0],  # L1 regularization to prevent overfitting
        'reg_lambda': [0, 0.1, 1.0]  # L2 regularization to prevent overfitting
    },
    
    'SVR': {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
        'kernel': ['rbf', 'linear', 'poly'],
        'epsilon': [0.01, 0.1, 0.2]
    },
    
    'KNN': {
        'n_neighbors': [15, 20, 25, 30, 35, 40],  # Increased n_neighbors range to address overfitting
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'p': [1, 2, 3],
        'leaf_size': [10, 30, 50, 100]  # Added leaf_size parameter for efficiency
    }
}

# Store tuned models and their results
tuned_models = {}
tuned_results = []

for name, model in models.items():
    if param_grids[name] is None:
        print(f"Skipping hyperparameter tuning for {name} (no parameters to tune)")
        # Still evaluate the model
        result = evaluate_model(model, X_train_scaled, y_train, X_test_scaled, y_test, name)
        tuned_models[name] = model
        tuned_results.append(result)
        continue
    
    print(f"Tuning hyperparameters for {name}...")
    
    # Create a fresh instance of the model
    model_to_tune = type(model)()
    
    # Set up randomized search
    n_iter = 30  # Number of parameter settings to try
    random_search = RandomizedSearchCV(
        estimator=model_to_tune,
        param_distributions=param_grids[name],
        n_iter=n_iter,
        scoring='r2',
        cv=5,
        verbose=1,
        random_state=42,
        n_jobs=-1
    )
    
    # Perform hyperparameter search
    random_search.fit(X_train_scaled, y_train)
    
    # Get best parameters
    print(f"Best parameters for {name}:")
    for param, value in random_search.best_params_.items():
        print(f"  {param}: {value}")
    
    # Get best model
    tuned_model = random_search.best_estimator_
    tuned_models[name] = tuned_model
    
    # Evaluate tuned model
    result = evaluate_model(tuned_model, X_train_scaled, y_train, X_test_scaled, y_test, name)
    tuned_results.append(result)
    
    print(f"  Test R²: {result['test_r2']:.4f}, RMSE: {result['test_rmse']:.2f}")

# Create DataFrame for tuned model results
tuned_results_df = pd.DataFrame([
    {
        'Model': r['model_name'],
        'Train R²': r['train_r2'],
        'Test R²': r['test_r2'],
        'Train RMSE': r['train_rmse'],
        'Test RMSE': r['test_rmse'],
        'Train MAE': r['train_mae'],
        'Test MAE': r['test_mae'],
        'Improvement over AOD-only (%)': (r['test_r2'] - aod_r2) / aod_r2 * 100 if aod_r2 > 0 else np.inf
    } for r in tuned_results
])

# Sort by test R² score
tuned_results_df = tuned_results_df.sort_values('Test R²', ascending=False).reset_index(drop=True)

# Save results for paper
tuned_results_df.to_csv('../results/tuned_model_comparison.csv', index=False)

# Display results
print("\nTuned Model Performance Comparison:")
print(tuned_results_df)

# 7. Check for overfitting and underfitting
print("\nChecking for overfitting and underfitting in tuned models...")

# Calculate the difference between train and test R²
tuned_results_df['R² Difference'] = tuned_results_df['Train R²'] - tuned_results_df['Test R²']
tuned_results_df['Overfitting Risk'] = tuned_results_df['R² Difference'].apply(
    lambda x: 'High' if x > 0.1 else ('Medium' if x > 0.05 else 'Low')
)

print("\nOverfitting analysis:")
print(tuned_results_df[['Model', 'Train R²', 'Test R²', 'R² Difference', 'Overfitting Risk']])

# Plot learning curves for models with high overfitting risk
models_to_check = tuned_results_df[tuned_results_df['Overfitting Risk'] != 'Low']['Model'].tolist()

if models_to_check:
    print(f"\nGenerating learning curves for models with potential overfitting: {models_to_check}")
    
    for model_name in models_to_check:
        model = tuned_models[model_name]
        
        # Generate learning curve data
        train_sizes = np.linspace(0.1, 1.0, 10)
        train_sizes, train_scores, test_scores = learning_curve(
            model, X_train_scaled, y_train, 
            train_sizes=train_sizes, cv=5, scoring='r2', n_jobs=-1
        )
        
        # Calculate mean and std for train/test scores
        train_mean = np.mean(train_scores, axis=1)
        train_std = np.std(train_scores, axis=1)
        test_mean = np.mean(test_scores, axis=1)
        test_std = np.std(test_scores, axis=1)
        
        # Plot learning curve
        plt.figure(figsize=(10, 6))
        plt.title(f'Learning Curve: {model_name}')
        plt.xlabel('Training Examples')
        plt.ylabel('R² Score')
        plt.grid()
        
        plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
        plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='orange')
        plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training Score')
        plt.plot(train_sizes, test_mean, 'o-', color='orange', label='Cross-validation Score')
        
        plt.legend(loc='best')
        plt.savefig(f'../figures/learning_curve_{model_name.replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Fix overfitting for tree-based models
        if model_name in ['Random Forest', 'Gradient Boosting', 'XGBoost']:
            print(f"\nAdjusting {model_name} to reduce overfitting...")
            
            if model_name == 'Random Forest':
                # Increase min_samples_leaf and reduce max_depth
                new_model = RandomForestRegressor(
                    n_estimators=model.n_estimators,
                    max_depth=min(model.max_depth, 15) if model.max_depth else 15,
                    min_samples_leaf=max(model.min_samples_leaf, 5),
                    min_samples_split=max(model.min_samples_split, 5),
                    max_features='sqrt',
                    random_state=42
                )
            
            elif model_name == 'Gradient Boosting':
                # Reduce learning rate and max_depth, increase min_samples_leaf
                new_model = GradientBoostingRegressor(
                    n_estimators=model.n_estimators,
                    learning_rate=min(model.learning_rate, 0.05),
                    max_depth=min(model.max_depth, 5),
                    min_samples_leaf=max(model.min_samples_leaf, 5),
                    min_samples_split=max(model.min_samples_split, 5),
                    subsample=0.8,
                    random_state=42
                )
            
            elif model_name == 'XGBoost':
                # Add regularization and reduce complexity
                new_model = XGBRegressor(
                    n_estimators=model.n_estimators,
                    learning_rate=min(model.learning_rate, 0.05),
                    max_depth=min(model.max_depth, 5),
                    min_child_weight=max(model.min_child_weight if hasattr(model, 'min_child_weight') else 1, 3),
                    subsample=0.8,
                    colsample_bytree=0.8,
                    reg_alpha=0.5,  # L1 regularization
                    reg_lambda=1.0,  # L2 regularization
                    random_state=42
                )
            
            # Evaluate adjusted model
            new_result = evaluate_model(new_model, X_train_scaled, y_train, X_test_scaled, y_test, f"{model_name} (Adjusted)")
            
            print(f"Original {model_name}:")
            print(f"  Train R²: {tuned_results_df[tuned_results_df['Model'] == model_name]['Train R²'].values[0]:.4f}")
            print(f"  Test R²: {tuned_results_df[tuned_results_df['Model'] == model_name]['Test R²'].values[0]:.4f}")
            print(f"  R² Difference: {tuned_results_df[tuned_results_df['Model'] == model_name]['R² Difference'].values[0]:.4f}")
            
            print(f"Adjusted {model_name}:")
            print(f"  Train R²: {new_result['train_r2']:.4f}")
            print(f"  Test R²: {new_result['test_r2']:.4f}")
            print(f"  R² Difference: {new_result['train_r2'] - new_result['test_r2']:.4f}")
            
            # If adjusted model performs better on test set or has less overfitting with similar performance
            if (new_result['test_r2'] > tuned_results_df[tuned_results_df['Model'] == model_name]['Test R²'].values[0] or
                (new_result['train_r2'] - new_result['test_r2'] < tuned_results_df[tuned_results_df['Model'] == model_name]['R² Difference'].values[0] and
                 new_result['test_r2'] >= tuned_results_df[tuned_results_df['Model'] == model_name]['Test R²'].values[0] - 0.01)):
                
                print(f"Replacing original {model_name} with adjusted version (better generalization)")
                tuned_models[model_name] = new_model
                
                # Update results
                for i, result in enumerate(tuned_results):
                    if result['model_name'] == model_name:
                        tuned_results[i] = new_result
                        break
                
                # Update DataFrame - Fix the KeyError by mapping column names correctly
                mapping = {
                    'Train R²': 'train_r2',
                    'Test R²': 'test_r2',
                    'Train RMSE': 'train_rmse',
                    'Test RMSE': 'test_rmse',
                    'Train MAE': 'train_mae',
                    'Test MAE': 'test_mae'
                }
                
                for df_col, result_key in mapping.items():
                    tuned_results_df.loc[tuned_results_df['Model'] == model_name, df_col] = new_result[result_key]
                
                tuned_results_df.loc[tuned_results_df['Model'] == model_name, 'R² Difference'] = new_result['train_r2'] - new_result['test_r2']
                tuned_results_df.loc[tuned_results_df['Model'] == model_name, 'Overfitting Risk'] = 'Low' if (new_result['train_r2'] - new_result['test_r2']) <= 0.05 else 'Medium'
            else:
                print(f"Keeping original {model_name} (better overall performance)")
        
        # Special handling for KNN which tends to overfit severely
        elif model_name == 'KNN':
            print(f"\nAdjusting KNN to reduce overfitting...")
            
            # Create a new KNN model with more neighbors to reduce overfitting
            new_model = KNeighborsRegressor(
                n_neighbors=max(model.n_neighbors, 40),  # Significantly increase neighbors
                weights='uniform',  # Use uniform weights to reduce overfitting
                metric='euclidean',
                leaf_size=50
            )
            
            # Evaluate adjusted model
            new_result = evaluate_model(new_model, X_train_scaled, y_train, X_test_scaled, y_test, "KNN (Adjusted)")
            
            print(f"Original KNN:")
            print(f"  Train R²: {tuned_results_df[tuned_results_df['Model'] == 'KNN']['Train R²'].values[0]:.4f}")
            print(f"  Test R²: {tuned_results_df[tuned_results_df['Model'] == 'KNN']['Test R²'].values[0]:.4f}")
            print(f"  R² Difference: {tuned_results_df[tuned_results_df['Model'] == 'KNN']['R² Difference'].values[0]:.4f}")
            
            print(f"Adjusted KNN:")
            print(f"  Train R²: {new_result['train_r2']:.4f}")
            print(f"  Test R²: {new_result['test_r2']:.4f}")
            print(f"  R² Difference: {new_result['train_r2'] - new_result['test_r2']:.4f}")
            
            # If adjusted model has less overfitting with similar or better test performance
            if new_result['train_r2'] - new_result['test_r2'] < tuned_results_df[tuned_results_df['Model'] == 'KNN']['R² Difference'].values[0]:
                print("Replacing original KNN with adjusted version (better generalization)")
                tuned_models['KNN'] = new_model
                
                # Update results
                for i, result in enumerate(tuned_results):
                    if result['model_name'] == 'KNN':
                        tuned_results[i] = new_result
                        break
                
                # Update DataFrame
                mapping = {
                    'Train R²': 'train_r2',
                    'Test R²': 'test_r2',
                    'Train RMSE': 'train_rmse',
                    'Test RMSE': 'test_rmse',
                    'Train MAE': 'train_mae',
                    'Test MAE': 'test_mae'
                }
                
                for df_col, result_key in mapping.items():
                    tuned_results_df.loc[tuned_results_df['Model'] == 'KNN', df_col] = new_result[result_key]
                
                tuned_results_df.loc[tuned_results_df['Model'] == 'KNN', 'R² Difference'] = new_result['train_r2'] - new_result['test_r2']
                tuned_results_df.loc[tuned_results_df['Model'] == 'KNN', 'Overfitting Risk'] = 'Low' if (new_result['train_r2'] - new_result['test_r2']) <= 0.05 else 'Medium'
            else:
                print("Keeping original KNN (better overall performance)")

# Check for underfitting (low R² on both train and test)
underfitting_models = tuned_results_df[(tuned_results_df['Train R²'] < 0.5) & (tuned_results_df['Test R²'] < 0.5)]['Model'].tolist()

if underfitting_models:
    print(f"\nModels with potential underfitting: {underfitting_models}")
    print("Consider increasing model complexity or feature engineering for these models.")

# 8. Visualize model comparison
plt.figure(figsize=(12, 8))
sns.barplot(x='Test R²', y='Model', data=tuned_results_df)
plt.axvline(x=aod_r2, color='red', linestyle='--', 
           label=f'AOD-only baseline (R²={aod_r2:.4f})')
plt.title('Tuned Model Comparison: Test R² Score')
plt.xlabel('R² Score')
plt.legend()
plt.grid(True)
plt.savefig('../figures/tuned_model_comparison_r2.png', dpi=300, bbox_inches='tight')
plt.show()

plt.figure(figsize=(12, 8))
sns.barplot(x='Test RMSE', y='Model', data=tuned_results_df)
plt.axvline(x=aod_rmse, color='red', linestyle='--', 
           label=f'AOD-only baseline (RMSE={aod_rmse:.2f})')
plt.title('Tuned Model Comparison: Test RMSE')
plt.xlabel('RMSE')
plt.legend()
plt.grid(True)
plt.savefig('../figures/tuned_model_comparison_rmse.png', dpi=300, bbox_inches='tight')
plt.show()

# 9. Visualize train vs test performance to identify overfitting
plt.figure(figsize=(14, 8))
models_df = tuned_results_df.copy()
models_df = pd.melt(models_df, 
                   id_vars=['Model'], 
                   value_vars=['Train R²', 'Test R²'],
                   var_name='Dataset', value_name='R² Score')

sns.barplot(x='Model', y='R² Score', hue='Dataset', data=models_df)
plt.title('Train vs Test R² Score Comparison')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../figures/train_vs_test_r2.png', dpi=300, bbox_inches='tight')
plt.show()

# 10. Select the best model based on test R²
best_result = tuned_results[tuned_results_df.iloc[0].name]
best_model = best_result['model']
best_model_name = best_result['model_name']

print(f"\nBest model: {best_model_name}")
print(f"Test R²: {best_result['test_r2']:.4f}")
print(f"Test RMSE: {best_result['test_rmse']:.2f}")
print(f"Improvement over AOD-only: {(best_result['test_r2'] - aod_r2) / aod_r2 * 100:.2f}%")
print(f"Overfitting risk: {tuned_results_df[tuned_results_df['Model'] == best_model_name]['Overfitting Risk'].values[0]}")

# 11. Visualize actual vs. predicted for best model
plt.figure(figsize=(12, 8))
plt.scatter(y_test, best_result['y_pred'], alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')

# Calculate correlation coefficient and add to plot
correlation = np.corrcoef(y_test, best_result['y_pred'])[0, 1]
plt.annotate(f'R = {correlation:.4f}', 
             xy=(0.05, 0.95), xycoords='axes fraction',
             fontsize=12, bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

plt.xlabel('Actual PM2.5 (μg/m³)')
plt.ylabel('Predicted PM2.5 (μg/m³)')
plt.title(f'Actual vs. Predicted PM2.5 Using {best_model_name}')
plt.grid(True)
plt.tight_layout()
plt.savefig('../figures/actual_vs_predicted.png', dpi=300, bbox_inches='tight')
plt.show()

# 12. Residual analysis
residuals = y_test - best_result['y_pred']

plt.figure(figsize=(12, 8))
plt.scatter(best_result['y_pred'], residuals, alpha=0.7)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted PM2.5 (μg/m³)')
plt.ylabel('Residuals')
plt.title('Residual Plot for Best Model')
plt.grid(True)
plt.tight_layout()
plt.savefig('../figures/residual_plot.png', dpi=300, bbox_inches='tight')
plt.show()

# Histogram of residuals
plt.figure(figsize=(12, 8))
sns.histplot(residuals, kde=True, bins=30)
plt.axvline(x=0, color='r', linestyle='--')
plt.xlabel('Residual Value')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.grid(True)
plt.tight_layout()
plt.savefig('../figures/residual_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# 13. Save best model for further analysis
model_info = {
    'model': best_model,
    'model_name': best_model_name,
    'features': features,
    'scaler': scaler,
    'performance': {
        'r2': best_result['test_r2'],
        'rmse': best_result['test_rmse'],
        'mae': best_result['test_mae']
    },
    'aod_only_performance': {
        'r2': aod_r2,
        'rmse': aod_rmse,
        'mae': aod_mae
    },
    'aod_model': aod_model
}

np.save('../models/best_model.npy', model_info, allow_pickle=True)
print("\nPart 3 completed. Best model saved for further analysis.")

In [ ]:
"""
PART 4: FEATURE IMPORTANCE & ADVANCED ANALYSIS
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance, partial_dependence, PartialDependenceDisplay
from sklearn.model_selection import train_test_split
import shap
import warnings
from scipy.stats import pearsonr, spearmanr
warnings.filterwarnings('ignore')

# Set plotting style for publication quality
plt.style.use('seaborn-whitegrid')
colors = plt.cm.tab10(np.linspace(0, 1, 10))

# 1. Load best model and data
print("Loading best model and data...")
model_info = np.load('../models/best_model.npy', allow_pickle=True).item()
best_model = model_info['model']
best_model_name = model_info['model_name']
features = model_info['features']
scaler = model_info['scaler']

processed_data = np.load('../data/processed_data.npy', allow_pickle=True).item()
X_train = processed_data['X_train']
X_test = processed_data['X_test']
y_train = processed_data['y_train']
y_test = processed_data['y_test']
X_train_scaled = processed_data['X_train_scaled']
X_test_scaled = processed_data['X_test_scaled']

# Check for NaN values and handle them
print("Checking for NaN values in the data...")
if X_test_scaled.isna().any().any() or y_test.isna().any():
    print("Found NaN values in test data. Removing rows with NaN values...")
    # Create a mask for non-NaN values
    mask = ~np.isnan(y_test) & ~np.isnan(X_test_scaled).any(axis=1)
    X_test_scaled = X_test_scaled[mask]
    X_test = X_test[mask]
    y_test = y_test[mask]
    print(f"After removing NaN values: {len(y_test)} samples remaining")

print(f"Best model: {best_model_name}")
print(f"Performance: R²={model_info['performance']['r2']:.4f}, RMSE={model_info['performance']['rmse']:.2f}")

# 2. Feature importance analysis based on model type
importances = None
if hasattr(best_model, 'feature_importances_'):
    # For tree-based models that have feature_importances_ attribute
    print("\nExtracting built-in feature importances...")
    importances = best_model.feature_importances_
    
    # Create dataframe of feature importances
    importance_df = pd.DataFrame({
        'Feature': features,
        'Importance': importances
    })
    
    # Sort by importance
    importance_df = importance_df.sort_values('Importance', ascending=False)
    
    # Save for paper
    importance_df.to_csv('../results/feature_importances.csv', index=False)
    
    # Plot feature importances
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=importance_df)
    plt.title(f'Feature Importance from {best_model_name}')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('../figures/feature_importances.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print top features
    print("\nTop 5 important features:")
    print(importance_df.head(5))
    
    # Find AOD importance and rank
    aod_importance = importance_df[importance_df['Feature'] == 'AOD']
    if not aod_importance.empty:
        aod_rank = importance_df[importance_df['Feature'] == 'AOD'].index[0] + 1
        print(f"\nAOD importance: {aod_importance['Importance'].values[0]:.4f}")
        print(f"AOD rank: {aod_rank} out of {len(features)}")
elif hasattr(best_model, 'coef_'):
    # For linear models that have coef_ attribute
    print("\nExtracting coefficients from linear model...")
    if len(best_model.coef_.shape) == 1:
        coeffs = best_model.coef_
    else:
        coeffs = best_model.coef_[0]
    
    # Create dataframe of coefficients
    coef_df = pd.DataFrame({
        'Feature': features,
        'Coefficient': coeffs,
        'Abs_Coefficient': np.abs(coeffs)
    })
    
    # Sort by absolute coefficient value
    coef_df = coef_df.sort_values('Abs_Coefficient', ascending=False)
    
    # Save for paper
    coef_df.to_csv('../results/model_coefficients.csv', index=False)
    
    # Plot coefficients
    plt.figure(figsize=(12, 8))
    colors = ['g' if c > 0 else 'r' for c in coef_df['Coefficient']]
    plt.barh(coef_df['Feature'], coef_df['Coefficient'], color=colors)
    plt.axvline(x=0, color='k', linestyle='--')
    plt.title('Model Coefficients (Green=Positive, Red=Negative)')
    plt.xlabel('Coefficient Value')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('../figures/model_coefficients.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print top coefficients
    print("\nTop 5 features by coefficient magnitude:")
    print(coef_df.head(5))
    
    # Find AOD coefficient and rank
    aod_coef = coef_df[coef_df['Feature'] == 'AOD']
    if not aod_coef.empty:
        aod_rank = coef_df[coef_df['Feature'] == 'AOD'].index[0] + 1
        print(f"\nAOD coefficient: {aod_coef['Coefficient'].values[0]:.4f}")
        print(f"AOD rank: {aod_rank} out of {len(features)}")

# 3. Permutation importance (model-agnostic approach)
print("\nCalculating permutation importance...")
try:
    perm_importance = permutation_importance(
        best_model, X_test_scaled, y_test, n_repeats=10, random_state=42, n_jobs=-1
    )

    # Create dataframe of permutation importances
    perm_importance_df = pd.DataFrame({
        'Feature': features,
        'Importance': perm_importance.importances_mean,
        'Std': perm_importance.importances_std
    })

    # Sort by importance
    perm_importance_df = perm_importance_df.sort_values('Importance', ascending=False)

    # Save for paper
    perm_importance_df.to_csv('../results/permutation_importance.csv', index=False)

    # Plot permutation importances
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=perm_importance_df, 
               xerr=perm_importance_df['Std'])
    plt.title('Permutation Feature Importance (Test Set)')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('../figures/permutation_importance.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Print top features by permutation importance
    print("\nTop 5 features by permutation importance:")
    print(perm_importance_df.head(5))

    # Find AOD permutation importance and rank
    aod_perm = perm_importance_df[perm_importance_df['Feature'] == 'AOD']
    if not aod_perm.empty:
        aod_perm_rank = perm_importance_df[perm_importance_df['Feature'] == 'AOD'].index[0] + 1
        print(f"\nAOD permutation importance: {aod_perm['Importance'].values[0]:.4f}")
        print(f"AOD permutation rank: {aod_perm_rank} out of {len(features)}")
except Exception as e:
    print(f"Error calculating permutation importance: {str(e)}")
    print("Creating empty permutation importance dataframe for downstream code")
    perm_importance_df = pd.DataFrame({
        'Feature': features,
        'Importance': np.zeros(len(features)),
        'Std': np.zeros(len(features))
    })
    perm_importance_df = perm_importance_df.sort_values('Importance', ascending=False)
    aod_perm_rank = 0

# 4. SHAP values for detailed feature impact (if model supports it)
try:
    print("\nCalculating SHAP values for feature interpretation...")
    # Create explainer
    explainer = shap.Explainer(best_model, X_train_scaled)
    
    # Calculate SHAP values on test set
    shap_values = explainer(X_test_scaled)
    
    # Summary plot
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values, X_test_scaled, plot_type="bar", show=False)
    plt.title('Feature Impact (SHAP Values)')
    plt.tight_layout()
    plt.savefig('../figures/shap_summary.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Detailed dependency plot for AOD
    plt.figure(figsize=(12, 8))
    shap.dependence_plot("AOD", shap_values.values, X_test_scaled, show=False)
    plt.title('SHAP Dependence Plot for AOD')
    plt.tight_layout()
    plt.savefig('../figures/shap_aod_dependence.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Save SHAP values for paper
    shap_df = pd.DataFrame({
        'Feature': features,
        'SHAP_abs_mean': np.abs(shap_values.values).mean(axis=0)
    })
    shap_df = shap_df.sort_values('SHAP_abs_mean', ascending=False)
    shap_df.to_csv('../results/shap_importance.csv', index=False)
    
    print("SHAP analysis completed successfully.")
except Exception as e:
    print(f"Could not compute SHAP values: {str(e)}")

# 5. Partial Dependence Plots for key features
print("\nGenerating Partial Dependence Plots...")
try:
    # Top 3 features from previous importance analysis
    if 'importance_df' in locals():
        top_features = importance_df.head(3)['Feature'].tolist()
    elif 'coef_df' in locals():
        top_features = coef_df.head(3)['Feature'].tolist()
    else:
        top_features = perm_importance_df.head(3)['Feature'].tolist()

    # Ensure AOD is included in the top features
    if 'AOD' not in top_features:
        top_features.append('AOD')

    # Get feature indices
    feature_indices = [list(features).index(feat) for feat in top_features]

    # Generate PDPs
    fig, ax = plt.subplots(figsize=(15, 10))
    PartialDependenceDisplay.from_estimator(
        best_model, X_test_scaled, feature_indices, 
        feature_names=top_features, ax=ax
    )
    plt.suptitle('Partial Dependence Plots for Top Features', fontsize=16)
    plt.tight_layout()
    plt.savefig('../figures/partial_dependence_plots.png', dpi=300, bbox_inches='tight')
    plt.show()
except Exception as e:
    print(f"Error generating Partial Dependence Plots: {str(e)}")

# 6. AOD-PM2.5 relationship analysis
print("\nAnalyzing AOD-PM2.5 relationship in detail...")
try:
    # Create AOD-only model (from loaded model info)
    aod_model = model_info['aod_model']
    aod_model_performance = model_info['aod_only_performance']

    # Compare full model vs. AOD-only model
    improvement = (model_info['performance']['r2'] - aod_model_performance['r2']) / aod_model_performance['r2'] * 100

    print(f"AOD-only model: R²={aod_model_performance['r2']:.4f}, RMSE={aod_model_performance['rmse']:.2f}")
    print(f"Full model: R²={model_info['performance']['r2']:.4f}, RMSE={model_info['performance']['rmse']:.2f}")
    print(f"Improvement with additional features: {improvement:.2f}%")

    # Visualize AOD-only vs full model predictions
    aod_preds = aod_model.predict(X_test[['AOD']].values.reshape(-1, 1))
    full_preds = best_model.predict(X_test_scaled)

    # Calculate correlations
    aod_only_pearson, _ = pearsonr(y_test, aod_preds)
    full_model_pearson, _ = pearsonr(y_test, full_preds)
    aod_only_spearman, _ = spearmanr(y_test, aod_preds)
    full_model_spearman, _ = spearmanr(y_test, full_preds)

    plt.figure(figsize=(12, 10))
    plt.scatter(y_test, aod_preds, alpha=0.5, label=f'AOD-only model (r={aod_only_pearson:.3f}, ρ={aod_only_spearman:.3f})', color='blue')
    plt.scatter(y_test, full_preds, alpha=0.5, label=f'{best_model_name} (r={full_model_pearson:.3f}, ρ={full_model_spearman:.3f})', color='red')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', label='1:1 line')
    plt.xlabel('Actual PM2.5 (μg/m³)')
    plt.ylabel('Predicted PM2.5 (μg/m³)')
    plt.title('AOD-only vs. Full Model Predictions')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('../figures/aod_vs_full_model.png', dpi=300, bbox_inches='tight')
    plt.show()
except Exception as e:
    print(f"Error in AOD-PM2.5 relationship analysis: {str(e)}")
    aod_only_pearson, full_model_pearson = 0, 0
    aod_only_spearman, full_model_spearman = 0, 0

# 7. PM2.5/AOD ratio analysis
print("\nAnalyzing PM2.5/AOD ratio...")
try:
    # Reload cleaned data to get the ratio
    df_cleaned = pd.read_csv('../data/cleaned_data.csv', parse_dates=['date'], index_col='date')

    # Calculate PM2.5/AOD ratio
    df_cleaned['PM25_AOD_Ratio'] = df_cleaned['PM25'] / df_cleaned['AOD']
    df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Basic statistics of the ratio
    ratio_stats = df_cleaned['PM25_AOD_Ratio'].describe()
    print("PM2.5/AOD Ratio Statistics:")
    print(ratio_stats)

    # Save ratio statistics for paper
    ratio_stats.to_csv('../results/pm25_aod_ratio_stats.csv')

    # Get best result from previous analysis
    try:
        best_result = np.load('../results/best_model_results.npy', allow_pickle=True).item()
    except:
        # If best_result is not available, create a placeholder
        print("Warning: best_result not found. Creating placeholder for downstream code.")
        best_result = {'y_pred': full_preds}

    # Add predicted PM2.5 to test data
    X_test_reset = X_test.reset_index(drop=True)
    test_data = pd.DataFrame({
        'Actual_PM25': y_test.reset_index(drop=True),
        'Predicted_PM25': best_result['y_pred'],
        'AOD': X_test_reset['AOD'],
    })
    
    # Calculate ratios
    test_data['Actual_Ratio'] = test_data['Actual_PM25'] / test_data['AOD']
    test_data['Predicted_Ratio'] = test_data['Predicted_PM25'] / test_data['AOD']
    test_data.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Calculate error in ratio prediction
    test_data['Ratio_Error'] = test_data['Predicted_Ratio'] - test_data['Actual_Ratio']
    test_data['Ratio_Pct_Error'] = (test_data['Ratio_Error'] / test_data['Actual_Ratio']) * 100

    # Calculate correlation for ratio prediction
    ratio_pearson, _ = pearsonr(test_data['Actual_Ratio'].dropna(), test_data['Predicted_Ratio'].dropna())
    ratio_spearman, _ = spearmanr(test_data['Actual_Ratio'].dropna(), test_data['Predicted_Ratio'].dropna())

    # Visualize actual vs predicted ratio
    plt.figure(figsize=(12, 8))
    plt.scatter(test_data['Actual_Ratio'], test_data['Predicted_Ratio'], alpha=0.7)
    plt.plot([test_data['Actual_Ratio'].min(), test_data['Actual_Ratio'].max()], 
             [test_data['Actual_Ratio'].min(), test_data['Actual_Ratio'].max()], 'r--')
    plt.xlabel('Actual PM2.5/AOD Ratio')
    plt.ylabel('Predicted PM2.5/AOD Ratio')
    plt.title(f'Actual vs. Predicted PM2.5/AOD Ratio (r={ratio_pearson:.3f}, ρ={ratio_spearman:.3f})')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('../figures/actual_vs_predicted_ratio.png', dpi=300, bbox_inches='tight')
    plt.show()
except Exception as e:
    print(f"Error in PM2.5/AOD ratio analysis: {str(e)}")
    ratio_stats = pd.Series({'mean': 0, '50%': 0, 'std': 0})
    ratio_pearson, ratio_spearman = 0, 0

# 8. Model performance by AOD range
print("\nAnalyzing model performance by AOD range...")
try:
    # Create AOD bins
    aod_bins = pd.cut(X_test['AOD'], bins=5)
    X_test_reset = X_test.reset_index(drop=True)
    y_test_reset = y_test.reset_index(drop=True)

    # Analysis by AOD bin
    aod_analysis = pd.DataFrame({
        'AOD': X_test_reset['AOD'],
        'Actual_PM25': y_test_reset,
        'Predicted_PM25': best_result['y_pred'],
        'Error': y_test_reset - best_result['y_pred'],
        'Pct_Error': (y_test_reset - best_result['y_pred']) / y_test_reset * 100,
        'AOD_bin': pd.cut(X_test_reset['AOD'], bins=5)
    })

    # Group by AOD bin
    aod_bin_analysis = aod_analysis.groupby('AOD_bin').agg({
        'AOD': ['count', 'mean'],
        'Actual_PM25': ['mean'],
        'Predicted_PM25': ['mean'],
        'Error': ['mean', 'std'],
        'Pct_Error': ['mean', 'std']
    })

    # Save for paper
    aod_bin_analysis.to_csv('../results/performance_by_aod_range.csv')

    print("Performance by AOD range:")
    print(aod_bin_analysis)

    # Visualize error by AOD bin
    plt.figure(figsize=(12, 8))
    sns.boxplot(x='AOD_bin', y='Error', data=aod_analysis)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('AOD Range')
    plt.ylabel('Prediction Error (μg/m³)')
    plt.title('Model Error by AOD Range')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('../figures/error_by_aod_range.png', dpi=300, bbox_inches='tight')
    plt.show()
except Exception as e:
    print(f"Error in model performance by AOD range analysis: {str(e)}")

# 9. Error Analysis: Where does the model fail?
print("\nIdentifying conditions where the model performs poorly...")
try:
    # Calculate absolute percentage error
    aod_analysis['Abs_Pct_Error'] = np.abs(aod_analysis['Pct_Error'])

    # Identify worst predictions (top 5% errors)
    threshold = np.percentile(aod_analysis['Abs_Pct_Error'].dropna(), 95)
    worst_predictions = aod_analysis[aod_analysis['Abs_Pct_Error'] > threshold]

    print(f"Analyzing {len(worst_predictions)} worst predictions (>95th percentile of error)")
    print(f"Error threshold: {threshold:.2f}% absolute percentage error")

    # Check if these errors are associated with particular AOD ranges
    print("\nAOD distribution in worst predictions:")
    print(worst_predictions['AOD'].describe())

    print("\nOverall AOD distribution:")
    print(aod_analysis['AOD'].describe())

    # Calculate correlation between AOD and actual PM2.5
    aod_pm25_pearson, _ = pearsonr(aod_analysis['AOD'], aod_analysis['Actual_PM25'])
    aod_pm25_spearman, _ = spearmanr(aod_analysis['AOD'], aod_analysis['Actual_PM25'])

    # Create scatter plot with worst predictions highlighted
    plt.figure(figsize=(12, 10))
    plt.scatter(aod_analysis['AOD'], aod_analysis['Actual_PM25'], alpha=0.5, 
               label='All predictions', color='blue')
    plt.scatter(worst_predictions['AOD'], worst_predictions['Actual_PM25'], alpha=0.8,
               label='Worst predictions', color='red')
    plt.xlabel('AOD')
    plt.ylabel('PM2.5 (μg/m³)')
    plt.title(f'Identifying Poor Model Performance Regions (AOD-PM2.5: r={aod_pm25_pearson:.3f}, ρ={aod_pm25_spearman:.3f})')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('../figures/worst_predictions.png', dpi=300, bbox_inches='tight')
    plt.show()
except Exception as e:
    print(f"Error in model failure analysis: {str(e)}")
    aod_pm25_pearson, aod_pm25_spearman = 0, 0

# 10. Generate summary table for paper
try:
    # Load cross-validation results if available
    try:
        cv_results_df = pd.read_csv('../results/cross_validation_results.csv')
    except:
        print("Warning: Cross-validation results not found. Using placeholder values.")
        cv_results_df = pd.DataFrame({
            'Model': [best_model_name],
            'R² (mean)': [model_info['performance']['r2']],
            'R² (std)': [0],
            'RMSE (mean)': [model_info['performance']['rmse']],
            'RMSE (std)': [0]
        })
    
    summary_table = pd.DataFrame({
        'Metric': [
            'Best Model', 
            'R² (Full Model)', 
            'RMSE (Full Model)', 
            'MAE (Full Model)',
            'R² (AOD-only Model)',
            'RMSE (AOD-only Model)',
            'Improvement from Meteorological Variables',
            'PM2.5/AOD Ratio (Mean)',
            'PM2.5/AOD Ratio (Median)',
            'PM2.5/AOD Ratio (Std)',
            'Top Feature',
            'AOD Importance Rank',
            'Sample Size',
            'R² from Cross-Validation',
            'RMSE from Cross-Validation',
            'Pearson Correlation (Full Model)',
            'Spearman Correlation (Full Model)',
            'Pearson Correlation (AOD-only Model)',
            'Spearman Correlation (AOD-only Model)'
        ],
        'Value': [
            best_model_name,
            f"{model_info['performance']['r2']:.4f}",
            f"{model_info['performance']['rmse']:.2f} μg/m³",
            f"{model_info['performance']['mae']:.2f} μg/m³",
            f"{model_info['aod_only_performance']['r2']:.4f}",
            f"{model_info['aod_only_performance']['rmse']:.2f} μg/m³",
            f"{improvement:.2f}%",
            f"{ratio_stats['mean']:.2f}",
            f"{ratio_stats['50%']:.2f}",
            f"{ratio_stats['std']:.2f}",
            perm_importance_df.iloc[0]['Feature'] if len(perm_importance_df) > 0 else "N/A",
            f"{aod_perm_rank} of {len(features)}" if aod_perm_rank > 0 else "N/A",
            f"{len(X_train) + len(X_test)}",
            f"{cv_results_df.iloc[0]['R² (mean)']:.4f} ± {cv_results_df.iloc[0]['R² (std)']:.4f}",
            f"{cv_results_df.iloc[0]['RMSE (mean)']:.2f} ± {cv_results_df.iloc[0]['RMSE (std)']:.2f}",
            f"{full_model_pearson:.4f}",
            f"{full_model_spearman:.4f}",
            f"{aod_only_pearson:.4f}",
            f"{aod_only_spearman:.4f}"
        ]
    })

    # Save summary for paper
    summary_table.to_csv('../results/analysis_summary.csv', index=False)

    print("\nAnalysis summary:")
    print(summary_table)
except Exception as e:
    print(f"Error generating summary table: {str(e)}")

print("\nPart 4 completed. Comprehensive analysis results saved for paper.")

In [ ]:
#############################################################
# Part 5: Advanced Visualization and Interpretation
#############################################################

print("## 5. Advanced Visualization and Interpretation")
print("Creating advanced visualizations to understand PM2.5 and AOD relationships...")

# Import additional libraries if needed
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Create figures directory if it doesn't exist
import os
if not os.path.exists('../figures'):
    os.makedirs('../figures')
    print("Created '../figures' directory for saving visualizations")

# 5.1 Basic visualizations of PM2.5 vs AOD relationship

# Scatterplot with regression line
plt.figure(figsize=(10, 8))
sns.regplot(x='AOD', y='PM25', data=df, scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
plt.title('Relationship between PM2.5 and AOD', fontsize=16)
plt.xlabel('AOD', fontsize=14)
plt.ylabel('PM2.5 (μg/m³)', fontsize=14)
plt.grid(True, alpha=0.3)

# Calculate correlation
corr = df['AOD'].corr(df['PM25'])
r2 = corr**2
plt.annotate(f"Correlation: {corr:.3f}\nR²: {r2:.3f}", 
             xy=(0.05, 0.92), xycoords='axes fraction',
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
             fontsize=12)

plt.tight_layout()
plt.savefig('../figures/pm25_aod_relationship.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Created PM2.5 vs AOD relationship plot")

# 5.2 Distribution of PM2.5 and AOD
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# PM2.5 distribution
sns.histplot(df['PM25'], kde=True, ax=ax1, color='skyblue')
ax1.set_title('Distribution of PM2.5 Concentrations', fontsize=14)
ax1.set_xlabel('PM2.5 (μg/m³)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.grid(True, alpha=0.3)

# Add summary statistics
pm_mean = df['PM25'].mean()
pm_median = df['PM25'].median()
pm_std = df['PM25'].std()
ax1.axvline(pm_mean, color='red', linestyle='--', label=f'Mean: {pm_mean:.2f}')
ax1.axvline(pm_median, color='green', linestyle='-.', label=f'Median: {pm_median:.2f}')
ax1.legend()

# AOD distribution
sns.histplot(df['AOD'], kde=True, ax=ax2, color='lightgreen')
ax2.set_title('Distribution of AOD Values', fontsize=14)
ax2.set_xlabel('AOD', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.grid(True, alpha=0.3)

# Add summary statistics
aod_mean = df['AOD'].mean()
aod_median = df['AOD'].median()
aod_std = df['AOD'].std()
ax2.axvline(aod_mean, color='red', linestyle='--', label=f'Mean: {aod_mean:.2f}')
ax2.axvline(aod_median, color='green', linestyle='-.', label=f'Median: {aod_median:.2f}')
ax2.legend()

plt.tight_layout()
plt.savefig('../figures/pm25_aod_distributions.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Created distribution plots for PM2.5 and AOD")

# 5.4 Meteorological factors influence
# Select meteorological features
met_features = [col for col in df.columns if col in 
                ['T2M', 'SP', 'U10', 'V10', 'BL', 'D2M']]

if len(met_features) > 0:
    # Correlation matrix
    corr_columns = ['PM25', 'AOD'] + met_features
    corr_matrix = df[corr_columns].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', 
              linewidths=0.5, vmin=-1, vmax=1, center=0)
    plt.title('Correlation Matrix: PM2.5, AOD, and Meteorological Factors', fontsize=16)
    plt.tight_layout()
    plt.savefig('../figures/correlation_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✓ Created correlation matrix heatmap")
    
    # 3D plot with most correlated meteorological factor
    if len(met_features) >= 1:
        # Find most correlated meteorological feature with PM2.5
        pm_corr = abs(corr_matrix['PM25'][met_features]).sort_values(ascending=False)
        top_met_feature = pm_corr.index[0]
        
        # Create 3D plot
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(111, projection='3d')
        
        scatter = ax.scatter(df['AOD'], df[top_met_feature], df['PM25'],
                           c=df[top_met_feature], cmap='viridis', 
                           s=30, alpha=0.7)
        
        ax.set_xlabel('AOD', fontsize=12)
        ax.set_ylabel(top_met_feature, fontsize=12)
        ax.set_zlabel('PM2.5 (μg/m³)', fontsize=12)
        ax.set_title(f'3D Relationship: PM2.5, AOD, and {top_met_feature}', fontsize=14)
        
        # Add colorbar
        cbar = plt.colorbar(scatter, ax=ax, shrink=0.7)
        cbar.set_label(top_met_feature, fontsize=12)
        
        # Rotate the plot for better viewing
        ax.view_init(elev=20, azim=45)
        
        plt.tight_layout()
        plt.savefig('../figures/3d_relationship.png', dpi=300, bbox_inches='tight')
        plt.show()
        print(f"✓ Created 3D visualization with {top_met_feature}")
else:
    print("No meteorological features found - skipping meteorological analysis")

# 5.5 Model performance visualization
if 'y_test' in locals() and 'y_pred' in locals():
    # Check if arrays are compatible for plotting
    if hasattr(y_test, 'values'):
        y_test_values = y_test.values
    else:
        y_test_values = y_test
        
    # Ensure shapes match
    if len(y_test_values) != len(y_pred):
        print(f"Warning: y_test ({len(y_test_values)}) and y_pred ({len(y_pred)}) have different lengths.")
        # Use the minimum length
        min_len = min(len(y_test_values), len(y_pred))
        y_test_values = y_test_values[:min_len]
        y_pred = y_pred[:min_len]
        print(f"Using first {min_len} elements for visualization")
        
    # Reshape if needed
    y_test_values = y_test_values.flatten() if hasattr(y_test_values, 'flatten') else y_test_values
    y_pred_flat = y_pred.flatten() if hasattr(y_pred, 'flatten') else y_pred
    
    # Create visualization of actual vs predicted values
    plt.figure(figsize=(10, 8))
    plt.scatter(y_test_values, y_pred_flat, alpha=0.5)
    
    # Get min and max for the reference line
    min_val = min(y_test_values.min(), y_pred_flat.min())
    max_val = max(y_test_values.max(), y_pred_flat.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')
    
    plt.xlabel('Actual PM2.5 (μg/m³)', fontsize=12)
    plt.ylabel('Predicted PM2.5 (μg/m³)', fontsize=12)
    plt.title('Actual vs Predicted PM2.5 Concentrations', fontsize=14)
    
    # Add performance metrics
    if 'all_model_results' in locals():
        # Find best model
        best_model_name = max(all_model_results, key=lambda x: x['test_r2'])['model_name']
        best_test_r2 = max(all_model_results, key=lambda x: x['test_r2'])['test_r2']
        best_test_rmse = max(all_model_results, key=lambda x: x['test_r2'])['test_rmse']
        best_test_mae = max(all_model_results, key=lambda x: x['test_r2'])['test_mae']
        
        plt.annotate(f"Model: {best_model_name}\nR²: {best_test_r2:.3f}\nRMSE: {best_test_rmse:.3f}\nMAE: {best_test_mae:.3f}", 
                   xy=(0.05, 0.92), xycoords='axes fraction',
                   bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
                   fontsize=12)
    
    plt.tight_layout()
    plt.savefig('../figures/actual_vs_predicted.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✓ Created actual vs predicted values plot")
    
    # Residual analysis
    residuals = y_test_values - y_pred_flat
    
    plt.figure(figsize=(15, 6))
    
    # Residual plot
    plt.subplot(1, 2, 1)
    plt.scatter(y_pred_flat, residuals, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted PM2.5 (μg/m³)', fontsize=12)
    plt.ylabel('Residuals', fontsize=12)
    plt.title('Residual Plot', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # Residual distribution
    plt.subplot(1, 2, 2)
    sns.histplot(residuals, kde=True, color='skyblue')
    plt.xlabel('Residual Value', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.title('Distribution of Residuals', fontsize=14)
    plt.grid(True, alpha=0.3)
    
    # Add residual statistics
    plt.annotate(f"Mean: {np.mean(residuals):.3f}\nStd Dev: {np.std(residuals):.3f}", 
               xy=(0.05, 0.92), xycoords='axes fraction',
               bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
               fontsize=12)
    
    plt.tight_layout()
    plt.savefig('../figures/residual_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✓ Created residual analysis plots")
    
    # Model performance comparison if available
    if 'all_model_results' in locals():
        # Filter out Ridge, Lasso, and ElasticNet models
        filtered_models = [m for m in all_model_results if not any(x in m['model_name'] for x in ['Ridge', 'Lasso', 'ElasticNet'])]
        
        # Sort models by test R² score
        sorted_models = sorted(filtered_models, key=lambda x: x['test_r2'], reverse=True)
        model_names = [m['model_name'] for m in sorted_models]
        test_r2 = [m['test_r2'] for m in sorted_models]
        test_rmse = [m['test_rmse'] for m in sorted_models]
        test_mae = [m['test_mae'] for m in sorted_models]
        
        plt.figure(figsize=(14, 10))
        
        # R² scores
        plt.subplot(3, 1, 1)
        bars = plt.bar(model_names, test_r2, color='skyblue')
        bars[0].set_color('darkblue')  # Highlight best model
        plt.title('Model Comparison - Test R² Score', fontsize=14)
        plt.ylabel('R² Score', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.3)
        
        # RMSE
        plt.subplot(3, 1, 2)
        bars = plt.bar(model_names, test_rmse, color='lightgreen')
        bars[0].set_color('darkgreen')  # Highlight best model
        plt.title('Model Comparison - Test RMSE', fontsize=14)
        plt.ylabel('RMSE', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.3)
        
        # MAE
        plt.subplot(3, 1, 3)
        bars = plt.bar(model_names, test_mae, color='salmon')
        bars[0].set_color('darkred')  # Highlight best model
        plt.title('Model Comparison - Test MAE', fontsize=14)
        plt.ylabel('MAE', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('../figures/model_comparison.png', dpi=300, bbox_inches='tight')
        plt.show()
        print("✓ Created model comparison plots")
else:
    print("Model predictions not found - skipping model performance visualization")

# 5.6 Feature importance visualization if available
if 'feature_importances' in locals() and 'feature_names' in locals():
    plt.figure(figsize=(10, 8))
    
    # Sort features by importance
    indices = np.argsort(feature_importances)[::-1]
    sorted_features = [feature_names[i] for i in indices]
    sorted_importances = feature_importances[indices]
    
    # Plot feature importances
    plt.barh(range(len(sorted_importances)), sorted_importances, align='center', color='lightblue')
    plt.yticks(range(len(sorted_importances)), sorted_features)
    plt.xlabel('Feature Importance', fontsize=12)
    plt.title('Feature Importance for PM2.5 Prediction', fontsize=14)
    plt.tight_layout()
    plt.savefig('../figures/feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✓ Created feature importance visualization")
elif 'best_model' in locals() and hasattr(best_model, 'feature_importances_'):
    # Try to extract feature importances from the best model if available
    feature_importances = best_model.feature_importances_
    feature_names = X_train.columns if hasattr(X_train, 'columns') else [f'Feature {i}' for i in range(X_train.shape[1])]
    
    plt.figure(figsize=(10, 8))
    
    # Sort features by importance
    indices = np.argsort(feature_importances)[::-1]
    sorted_features = [feature_names[i] for i in indices]
    sorted_importances = feature_importances[indices]
    
    # Plot feature importances
    plt.barh(range(len(sorted_importances)), sorted_importances, align='center', color='lightblue')
    plt.yticks(range(len(sorted_importances)), sorted_features)
    plt.xlabel('Feature Importance', fontsize=12)
    plt.title('Feature Importance for PM2.5 Prediction', fontsize=14)
    plt.tight_layout()
    plt.savefig('../figures/feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✓ Created feature importance visualization from best model")
else:
    print("Feature importance information not found - skipping feature importance visualization")

print("\n✅ Part 5 Completed: Advanced visualization and interpretation completed successfully")
print("All visualizations have been saved to the '../figures' directory")

In [ ]:
#############################################################
# Part 6: Advanced Model Interpretation & Conclusions
#############################################################

print("## 6. Advanced Model Interpretation & Conclusions")
print("Implementing advanced model interpretation techniques and drawing final conclusions...")

# Import additional libraries if needed
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

# Try to import SHAP for advanced model interpretation
try:
    import shap
    shap_available = True
    print("✓ SHAP library available for model interpretation")
except ImportError:
    shap_available = False
    print("⚠️ SHAP library not found. Install with 'pip install shap' for advanced model interpretation")

# Create results directory if it doesn't exist
if not os.path.exists('../results'):
    os.makedirs('../results')
    print("Created '../results' directory for saving analysis results")

# 6.1 Advanced Model Interpretation with SHAP values
if 'best_model' in locals() and 'X_test' in locals() and shap_available:
    print("\n6.1 Generating SHAP values for model interpretation...")
    
    # Check if X_test is scaled
    X_for_shap = X_test_scaled if 'X_test_scaled' in locals() else X_test
    
    # Convert to DataFrame if not already
    if not isinstance(X_for_shap, pd.DataFrame):
        try:
            X_for_shap = pd.DataFrame(X_for_shap, columns=feature_names)
        except:
            feature_names = [f"Feature_{i}" for i in range(X_for_shap.shape[1])]
            X_for_shap = pd.DataFrame(X_for_shap, columns=feature_names)
    
    # Create a prediction function that SHAP can use
    def model_predict(X):
        return best_model.predict(X)
    
    # Use KernelExplainer which works with any model
    try:
        # Choose a subset of data for background distribution (for efficiency)
        background = shap.sample(X_for_shap, 100)
        
        # Create explainer using KernelExplainer which works with any model
        explainer = shap.KernelExplainer(model_predict, background)
        
        # Calculate SHAP values for a subset of test data (for efficiency)
        sample_size = min(100, len(X_for_shap))
        shap_values = explainer.shap_values(X_for_shap.iloc[:sample_size])
        
        # Summary plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_for_shap.iloc[:sample_size], plot_type="bar", show=False)
        plt.title("Feature Importance (SHAP Values)", fontsize=16)
        plt.tight_layout()
        plt.savefig('../figures/shap_feature_importance.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Detailed SHAP plots
        plt.figure(figsize=(14, 10))
        shap.summary_plot(shap_values, X_for_shap.iloc[:sample_size], show=False)
        plt.title("SHAP Summary Plot", fontsize=16)
        plt.tight_layout()
        plt.savefig('../figures/shap_summary.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Dependence plots for top 3 features (by average absolute SHAP value)
        feature_importance = np.abs(shap_values).mean(axis=0)
        top_indices = np.argsort(feature_importance)[-3:]
        
        for idx in top_indices:
            feature_name = X_for_shap.columns[idx]
            plt.figure(figsize=(10, 6))
            shap.dependence_plot(
                idx, shap_values, X_for_shap.iloc[:sample_size],
                feature_names=list(X_for_shap.columns), show=False
            )
            plt.title(f"SHAP Dependence Plot for {feature_name}", fontsize=14)
            plt.tight_layout()
            plt.savefig(f'../figures/shap_dependence_{feature_name}.png', dpi=300, bbox_inches='tight')
            plt.show()
        
        print("✓ SHAP analysis completed successfully")
    except Exception as e:
        print(f"⚠️ Error generating SHAP values: {e}")
        print("Continuing with alternative model interpretation methods...")
else:
    print("⚠️ SHAP analysis skipped (missing model, test data, or SHAP library)")

# 6.2 Partial Dependence Plots (PDPs) for interpreting feature effects
if 'best_model' in locals() and 'X_test' in locals():
    print("\n6.2 Creating Partial Dependence Plots...")
    
    try:
        from sklearn.inspection import PartialDependenceDisplay
        
        # Get feature names
        if 'X_test_scaled' in locals() and hasattr(X_test_scaled, 'columns'):
            feature_names = X_test_scaled.columns
        elif hasattr(X_test, 'columns'):
            feature_names = X_test.columns
        else:
            feature_names = [f"Feature_{i}" for i in range(X_test.shape[1])]
        
        # Determine top features - prioritize feature importance if available
        if 'feature_importances' in locals():
            top_indices = np.argsort(feature_importances)[-4:]
        elif hasattr(best_model, 'feature_importances_'):
            top_indices = np.argsort(best_model.feature_importances_)[-4:]
        else:
            # Without feature importances, use first few features
            top_indices = range(min(4, len(feature_names)))
        
        # Create PDPs for top features
        X_for_pdp = X_test_scaled if 'X_test_scaled' in locals() else X_test
        
        # Generate PDPs
        fig, ax = plt.subplots(figsize=(14, 8))
        display = PartialDependenceDisplay.from_estimator(
            best_model, X_for_pdp, top_indices, 
            feature_names=feature_names,
            ax=ax, kind='average'
        )
        plt.suptitle('Partial Dependence Plots for Top Features', fontsize=16)
        plt.tight_layout()
        plt.savefig('../figures/partial_dependence_plots.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("✓ Partial Dependence Plots created successfully")
    except Exception as e:
        print(f"⚠️ Error creating Partial Dependence Plots: {e}")

# Save the best model if available
if 'best_model' in locals():
    try:
        import joblib
        
        # Create models directory if it doesn't exist
        if not os.path.exists('../models'):
            os.makedirs('../models')
            print("Created '../models' directory for saving trained models")
        
        # Save the model
        model_filename = f"../models/best_model_{best_model_name if 'best_model_name' in locals() else 'final'}.joblib"
        joblib.dump(best_model, model_filename)
        print(f"✓ Best model saved to '{model_filename}'")
        
        # Save the scaler if available
        if 'scaler' in locals():
            scaler_filename = "../models/scaler.joblib"
            joblib.dump(scaler, scaler_filename)
            print(f"✓ Scaler saved to '{scaler_filename}'")
    except Exception as e:
        print(f"⚠️ Error saving model: {e}")

print("\n✅ Part 6 Completed: Advanced model interpretation completed")